# Part B: Person Following with Obstacle Avoidance

This notebook implements intelligent person following that:
- Locks onto and tracks a specific person
- Avoids obstacles (other people and objects) using YOLO detection
- Maintains safe distance while following
- Navigates around obstacles to continue following
- Handles occlusions (when target person temporarily disappears)
- Has emergency depth-based collision avoidance

**Keyboard controls:**
- 'g' = Go (start following first detected person)
- 's' = Stop (halt robot and reset)

## Step 1: Display Widgets

Standard widget setup for displaying color and depth images side by side.

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create two widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

## Step 2: Load YOLO Model

Load the TensorRT YOLO model for detecting humans and objects.

In [ ]:
from ultralytics import YOLO

# Load YOLO11 TensorRT model
model = YOLO("yolo11l_half.engine")
print("YOLO model loaded successfully")

## Step 3: Person Re-identification System

This class uses HSV color histograms to track the same person across frames.
When a person is first detected, their clothing colors are saved.
In future frames, detected people are matched against these stored patterns.

In [ ]:
import numpy as np

class HistogramPersonIdentifier:
    """
    Person re-identification using HSV histogram matching.
    
    How it works:
    1. Extract clothing region from person's bounding box
    2. Convert to HSV color space (more robust than RGB)
    3. Calculate histogram of Hue and Saturation channels
    4. Compare new detections against stored histograms
    5. Assign consistent person_id if match found
    """

    def __init__(self, similarity_thresh=0.4, h_bins=16, s_bins=16):
        """
        Args:
            similarity_thresh: Correlation score threshold (0-1)
                              Lower = more lenient matching
            h_bins: Number of Hue histogram bins
            s_bins: Number of Saturation histogram bins
        """
        self.person_db = {}  # person_id -> histogram
        self.next_id = 0
        self.similarity_thresh = similarity_thresh
        self.h_bins = h_bins
        self.s_bins = s_bins

    def _get_histogram(self, roi_bgr):
        """Compute normalized HSV histogram for a region of interest."""
        hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
        # Calculate 2D histogram (Hue and Saturation only, ignore Value)
        hist = cv2.calcHist(
            [hsv], [0, 1], None,
            [self.h_bins, self.s_bins],
            [0, 180, 0, 256]  # H range: 0-180, S range: 0-256
        )
        # Normalize to 0-1 range for comparison
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        return hist

    def _match_person(self, hist):
        """Find best matching person ID from database, or None if no good match."""
        best_id = None
        best_score = -1

        # Compare against all known people
        for pid, ref_hist in self.person_db.items():
            # Correlation comparison (higher = more similar)
            score = cv2.compareHist(ref_hist, hist, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score = score
                best_id = pid

        # Return ID only if similarity is above threshold
        if best_score > self.similarity_thresh:
            return best_id
        return None

    def assign_ids(self, frame, detections):
        """
        Assign stable person IDs to all detected humans in frame.
        
        Returns:
            List of tuples: (x1, y1, x2, y2, person_id)
        """
        results = []

        for box in detections.boxes:
            cls = int(box.cls[0])
            if cls != 0:  # Only process PERSON class (class 0)
                continue

            # Extract bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            roi = frame[y1:y2, x1:x2]

            if roi.size == 0:
                continue

            # Get histogram for this detection
            hist = self._get_histogram(roi)
            pid = self._match_person(hist)

            # Create new identity if no match found
            if pid is None:
                pid = self.next_id
                self.person_db[pid] = hist
                self.next_id += 1

            results.append((x1, y1, x2, y2, pid))

        return results

# Initialize the identifier
identifier = HistogramPersonIdentifier(similarity_thresh=0.4)
print("Person identifier initialized")

## Step 4: Camera System Initialization

Initialize ZED2i camera with:
- VGA resolution (672x376) for faster processing
- ULTRA depth mode for accurate distance measurements
- Threaded capture for real-time performance

In [ ]:
import traitlets
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize robot motor control
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    """
    Camera class with traitlets for real-time updates.
    Captures color and depth images in separate thread.
    """
    color_value = traitlets.Any()  # Triggers callback when new frame arrives
    
    def __init__(self):
        super(Camera, self).__init__()

        # Initialize ZED camera
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open:", repr(status), "Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        # Get camera resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        
        print(f"Camera initialized: {self.width}x{self.height}")

    def _capture_frames(self):
        """Threaded capture loop - runs continuously."""
        while self.thread_runnning_flag:
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve images
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                
                # Convert BGRA to BGR
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                # Get depth as numpy array
                self.depth_image = np.asanyarray(self.depth.get_data())

    def start(self):
        """Start the capture thread."""
        if not self.thread_runnning_flag:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()
            print("Camera thread started")

    def stop(self):
        """Stop the capture thread and motors."""
        if self.thread_runnning_flag:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()
            print("Camera stopped")

# Initialize and start camera
camera = Camera()
camera.start()

## Step 5: Detection and Navigation Functions

This cell contains all the core logic:
1. **detect_target_human()** - Finds and tracks the target person
2. **detect_obstacles_yolo()** - Identifies obstacles using YOLO (other people + objects)
3. **check_depth_emergency()** - Emergency collision avoidance using depth
4. **smart_follow_with_avoidance()** - Main navigation logic

In [ ]:
# Global variables
target_person_id = None  # ID of person we're following
frames_without_target = 0  # Counter for lost target

def detect_target_human(frame):
    """
    Detect and track the target person.
    
    On first detection: Locks onto first person seen
    Subsequent frames: Searches for that specific person_id
    
    Returns:
        (human_detected, distance, bbox, person_id)
        - human_detected: bool
        - distance: float (mm) or inf
        - bbox: [x1, y1, x2, y2] or None
        - person_id: int or None
    """
    global target_person_id, identifier
    
    # Run YOLO detection
    results = model(frame, verbose=False)[0]
    
    # Get person IDs from histogram matching
    tracks = identifier.assign_ids(frame, results)
    
    if len(tracks) == 0:
        return False, float('inf'), None, target_person_id
    
    # LOCK ONTO FIRST PERSON if not already locked
    if target_person_id is None:
        target_person_id = tracks[0][4]  # Get person_id from first detection
        print(f"🎯 LOCKED onto person ID: {target_person_id}")
    
    # FIND TARGET PERSON in current detections
    target_found = None
    for x1, y1, x2, y2, pid in tracks:
        if pid == target_person_id:
            target_found = (x1, y1, x2, y2, pid)
            break
    
    # Target not in current frame
    if target_found is None:
        return False, float('inf'), None, target_person_id
    
    # Calculate distance to target
    x1, y1, x2, y2, pid = target_found
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    
    # Get depth at center of bounding box
    distance = camera.depth_image[center_y, center_x]
    
    # Handle invalid depth readings
    if np.isnan(distance) or distance <= 0:
        distance = float('inf')
    
    bbox = [x1, y1, x2, y2]
    return True, distance, bbox, target_person_id


def detect_obstacles_depth(depth_image, human_bbox=None, min_safe_distance=400):
    """
    Detect obstacles using depth image (Tutorial 3 Part B approach).
    Excludes the region where target human is located.
    
    Returns:
        dict with 'obstacle_detected' (bool) and 'min_distance' (float)
    """
    depth = np.nan_to_num(depth_image.copy(), nan=0.0).astype(np.float32)
    
    # Define central area (from Tutorial 3 Part B)
    depth[:94, :] = 0      # Top region
    depth[282:, :] = 0     # Bottom region  
    depth[:, :168] = 0     # Left region
    depth[:, 504:] = 0     # Right region
    
    # Filter depth values (from Tutorial 3 Part B)
    depth[depth < 100] = 0    # Too close noise
    depth[depth > 1000] = 0   # Too far / unreliable
    
    # If we have target human bbox, exclude that region from obstacle detection
    if human_bbox is not None:
        x1, y1, x2, y2 = map(int, human_bbox)
        # Add margin around human bbox
        margin = 30
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(depth.shape[1], x2 + margin)
        y2 = min(depth.shape[0], y2 + margin)
        depth[y1:y2, x1:x2] = 0  # Zero out human's region
    
    # Check if any obstacles remain
    if depth.max() == 0:
        return {'obstacle_detected': False, 'min_distance': float('inf')}
    
    # Find minimum distance in remaining valid regions
    valid_depths = depth[depth != 0]
    min_distance = valid_depths.min()
    
    # Check if obstacle is within danger zone
    if min_distance < min_safe_distance:
        return {'obstacle_detected': True, 'min_distance': float(min_distance)}
    
    return {'obstacle_detected': False, 'min_distance': float(min_distance)}


def check_depth_emergency(depth_image, emergency_threshold=300):
    """
    Emergency collision avoidance using depth image.
    Catches obstacles that YOLO might miss (poles, wires, etc.)
    
    Args:
        emergency_threshold: Distance in mm for emergency stop
    
    Returns:
        dict with 'emergency' (bool) and 'distance' (float)
    """
    depth = np.nan_to_num(depth_image.copy(), nan=0).astype(np.float32)
    
    # Check center region directly in front of robot
    h, w = depth.shape
    center_region = depth[h//3:2*h//3, w//3:2*w//3]
    
    # Filter for valid depths in danger zone
    valid = center_region[(center_region > 50) & (center_region < emergency_threshold)]
    
    if valid.size > 0:
        return {
            'emergency': True,
            'distance': float(valid.min())
        }
    
    return {'emergency': False, 'distance': float('inf')}


def smart_follow_with_avoidance(frame, depth_colormap, human_detected, 
                                 human_distance, human_bbox, obstacle_info):
    """
    Main navigation logic with simplified depth-based obstacle avoidance.
    """
    global frames_without_target
    
    if not human_detected:
        # Target lost - search behavior
        frames_without_target += 1
        robot.spinRight(0.2)
        cv2.putText(depth_colormap, f'SEARCHING... ({frames_without_target} frames)', 
                   (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        return
    
    frames_without_target = 0
    
    # Calculate human's position
    human_center_x = (human_bbox[0] + human_bbox[2]) / 2
    frame_center_x = camera.width / 2
    human_offset = human_center_x - frame_center_x
    
    # OBSTACLE DETECTED (from Tutorial 3 Part B pattern)
    if obstacle_info['obstacle_detected']:
        obs_distance = obstacle_info['min_distance']
        
        if obs_distance < 300:
            # Very close - backup and turn away
            robot.backward(0.3)
            cv2.putText(depth_colormap, f'OBSTACLE! {obs_distance:.0f}mm - BACKING UP', 
                       (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        elif obs_distance < 400:
            # Close - turn toward human's direction
            if abs(human_offset) > 50:
                if human_offset < 0:
                    robot.left(0.3)
                else:
                    robot.right(0.3)
                cv2.putText(depth_colormap, f'AVOIDING {obs_distance:.0f}mm - TURNING', 
                           (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2)
            else:
                # Human centered but blocked - default turn left
                robot.left(0.3)
                cv2.putText(depth_colormap, f'BLOCKED {obs_distance:.0f}mm - NAVIGATING', 
                           (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2)
        else:
            # Moderate distance - slow approach
            robot.forward(0.2)
            cv2.putText(depth_colormap, f'CAUTIOUS {obs_distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 0), 2)
    
    else:
        # NO OBSTACLES - Normal following (same as before)
        if human_distance < 450:
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'Too close {human_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        elif human_distance >= 450 and human_distance <= 600:
            robot.stop()
            cv2.putText(depth_colormap, f'Perfect {human_distance:.0f}mm <3', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        
        elif human_distance > 600 and human_distance <= 1200:
            if abs(human_offset) > 50:
                if human_offset < 0:
                    robot.left(0.3)
                else:
                    robot.right(0.3)
            else:
                robot.forward(0.3)
            cv2.putText(depth_colormap, f'Approaching {human_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
        
        else:
            if abs(human_offset) > 50:
                if human_offset < 0:
                    robot.left(0.4)
                else:
                    robot.right(0.4)
            else:
                robot.forward(0.5)
            cv2.putText(depth_colormap, f'Coming to you {human_distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)            cv2.putText(depth_colormap, f'Coming to you {human_distance:.0f}mm', (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)

print("Detection and navigation functions loaded")

## Step 6: Main Control Loop with Keyboard Input

Integrates all components:
- Keyboard control (g=go, s=stop)
- Target person detection and tracking
- Obstacle detection (YOLO + depth)
- Smart navigation with visual feedback

In [ ]:
# Global behavior state
current_behavior = 'stop'

# Create keyboard input widget
text_input = widgets.Text(
    value='',
    placeholder='Type: g=Go, s=Stop',
    description='Control:',
    disabled=False
)

def on_text_change(change):
    """Handle keyboard input for behavior control."""
    global current_behavior, target_person_id
    
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()
        
        if last_char == 'g':
            current_behavior = 'go'
            print('\n🚀 STARTED - Robot will follow first detected person')
            
        elif last_char == 's':
            current_behavior = 'stop'
            target_person_id = None  # Reset target
            robot.stop()
            print('\nSTOPPED - Robot halted, target reset')

text_input.observe(on_text_change, names='value')
display(text_input)


def main_callback(change):
    """
    Main control callback - executed every frame.
    
    Processing pipeline:
    1. Get current frame
    2. Detect target human (with person ID tracking)
    3. Detect obstacles (other humans + objects)
    4. Check for emergency collision
    5. Execute appropriate behavior
    6. Visualize everything
    """
    global current_behavior, target_person_id
    
    # Get updated frame from camera
    frame = change['new']
    depth_image = camera.depth_image
    
    # Create depth colormap for visualization
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    # 1. DETECT TARGET HUMAN
    human_detected, human_distance, human_bbox, target_id = detect_target_human(frame)
    
    # 2. DETECT OBSTACLES (only if we have a target)
    obstacle_info = detect_obstacles_depth(depth_image, human_bbox if human_detected else None, min_safe_distance=400)

    
    # 3. EMERGENCY DEPTH CHECK
    emergency = check_depth_emergency(depth_image, emergency_threshold=250)
    
    # 4. VISUALIZE DETECTIONS
    
    # Draw target human (green box)
    if human_detected and human_bbox:
        x1, y1, x2, y2 = map(int, human_bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(frame, f'TARGET (ID:{target_id}) {human_distance:.0f}mm', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    
    # 5. BEHAVIOR EXECUTION
    
    if current_behavior == 'stop':
        # Stopped state
        robot.stop()
        cv2.putText(depth_colormap, '🛑 STOPPED', (220, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (128, 128, 128), 2)
        cv2.putText(frame, 'Press G to start following', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    elif emergency['emergency']:
        # EMERGENCY STOP - highest priority
        robot.stop()
        cv2.putText(depth_colormap, f'🚨 EMERGENCY STOP {emergency["distance"]:.0f}mm!', (100, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 3)
        cv2.putText(frame, 'EMERGENCY - Object too close!', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    elif current_behavior == 'go':
        # ACTIVE FOLLOWING
        smart_follow_with_avoidance(frame, depth_colormap, human_detected, human_distance, human_bbox, obstacle_info)
    
    # Display current state on frame
    status_text = f'Mode: {current_behavior.upper()}'
    if target_id is not None:
        status_text += f' | Target ID: {target_id}'
    
    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    # 6. DISPLAY IMAGES
    scale = 0.3  # Scale down for network efficiency
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)


# Attach callback to camera updates
camera.observe(main_callback, names=['color_value'])

print('\n' + '='*50)
print('🤖 PERSON FOLLOWING SYSTEM READY')
print('='*50)
print('\nKeyboard Controls:')
print('  g = GO (start following first detected person)')
print('  s = STOP (halt robot and reset target)')
print('='*50)